In [54]:
import get_xbox
import set_database
import goldmine
import set_database
import get_metadata
import datetime

In [55]:
class GameObject:
    """ The purpose of this class is to implement all other available modules which help construct game
        information and collect metadata prior to exporting this to a database.

        The only information it assumes to be supplied is the game name and platform (optional),
        and should resolve to an error if this is incorrect.
    """

    def __init__(self, game_name, game_platform=None):
        self.game_name = game_name
        self.game_rating = None
        self.game_img_loc = None
        self.game_genres = None
        self.game_platform = game_platform
        self.game_cost_at_store = None
        self.game_found = False
        self.record_created = datetime.datetime.now().strftime("%d/%m/%Y")
        self.meta_critic_urls = None
        self.fetch_failed = False

    def set_metadata(self):
        # format game name for applicable urls at metacritic.com
        fmt_game_name = self.game_name.replace(':', "")
        fmt_game_name = fmt_game_name.replace(" ", "-")
        fmt_game_name = fmt_game_name.lower()

        # send request to metacritic to get rating/
        data = get_metadata.get_metadata(fmt_game_name)

        # set the objects values to return values from metacritic
        self.game_genres = list(data[1])

        # cycle through tested platforms and find valid platforms
        temp_list = []
        for platform in data[0]:
            if data[0][platform]:
                temp_list.append(platform)
        self.game_platform = temp_list

        # game rating
        self.game_rating = data[0]

        # send request to microsoft to get artwork / price at store
        msoft_img_price = get_xbox.get_xbox_details(self.game_name)
        
        # initialise msoft_img_price_lower
        msoft_img_price_lower = None
        
        # below occaisionally fails, but usually not twice in a row. If fail, try again
        try:
            msoft_img_price_lower = dict((key.lower(), value) for key, value in msoft_img_price.items())
        except AttributeError:
            self.fetch_failed = True
            print("Error occurred with initial get_xbox_details query, thus except invalidated. Please send the"
                  "request again")
        
        if msoft_img_price_lower != None:
            for key in msoft_img_price_lower:
                if self.game_name in key:
                    # print(msoft_img_price_lower.get(key))
                    self.game_img_loc = msoft_img_price_lower.get(key)[0]
                    try:
                        self.game_img_loc = self.game_img_loc.replace("h=300&w=200", "w=720")
                    except:
                        pass
                    self.game_cost_at_store = msoft_img_price_lower.get(key)[1]
                    self.game_name = key
                    break
        else:
            print("The dictionary is empty.")            

        try:
            self.game_name = self.game_name.strip(" pre-order")
        except:
            pass

        # metacritic urls
        self.meta_critic_urls = data[2]

    def get_game_data(self):
        print(self.game_name)
        print(self.game_rating)
        print(self.game_img_loc)
        print(self.game_genres)
        print(self.game_platform)
        print(self.game_cost_at_store)
        print(self.record_created)
        print(self.meta_critic_urls)
        return self.game_name, self.game_rating, self.game_img_loc, self.game_genres, self.game_platform,\
            self.game_cost_at_store, self.record_created, self.meta_critic_urls

    def queries(self):
        query_list = []
        insert_game_name = """ INSERT INTO games_database\
        (game_id, game_name, game_trailer, date_recorded)\
        VALUES (NULL,%s,%s,%s); """
        vals = (self.game_name, "www.youtube.com", self.record_created)
        query_list.append(insert_game_name)
        print(query_list)

        for query in query_list:
            set_database.send_to_db(query,vals)

In [56]:
fifa = GameObject("borderlands 3")


In [57]:
fifa.set_metadata()

In [58]:
fifa.get_game_data()

borderlands 3
{'Xbox One': 'Rating not currently available.', 'Playstation 4': 'Rating not currently available.', 'PC': 'Rating not currently available.', 'Nintendo Switch': False}
//store-images.s-microsoft.com/image/apps.29198.64533163757722640.6f7413b4-c2c4-4451-8064-60774a6d143e.e0f7a7f8-ab7d-458a-a4fc-2bfc8a8fbfcc
['First-Person', 'Shooter', 'Action', 'Arcade']
['Xbox One', 'Playstation 4', 'PC']
€69.99
11/07/2019
{'Xbox One': 'https://www.metacritic.com/game/xbox-one/borderlands-3', 'Playstation 4': 'https://www.metacritic.com/game/playstation-4/borderlands-3', 'PC': 'https://www.metacritic.com/game/pc/borderlands-3'}


('borderlands 3',
 {'Xbox One': 'Rating not currently available.',
  'Playstation 4': 'Rating not currently available.',
  'PC': 'Rating not currently available.',
  'Nintendo Switch': False},
 '//store-images.s-microsoft.com/image/apps.29198.64533163757722640.6f7413b4-c2c4-4451-8064-60774a6d143e.e0f7a7f8-ab7d-458a-a4fc-2bfc8a8fbfcc',
 ['First-Person', 'Shooter', 'Action', 'Arcade'],
 ['Xbox One', 'Playstation 4', 'PC'],
 '€69.99',
 '11/07/2019',
 {'Xbox One': 'https://www.metacritic.com/game/xbox-one/borderlands-3',
  'Playstation 4': 'https://www.metacritic.com/game/playstation-4/borderlands-3',
  'PC': 'https://www.metacritic.com/game/pc/borderlands-3'})

In [59]:
fifa.queries()

[' INSERT INTO games_database        (game_id, game_name, game_trailer, date_recorded)        VALUES (NULL,%s,%s,%s); ']


TypeError: send_to_db() takes 1 positional argument but 2 were given